### vi. Attacker relationship (how do they claim to know the victim, 1) met online? 2) friend of a friend?, 3) met the victim in person before)

#### Example sentences:
- I **got your contact during** one of my recent official trips to S/korea March/April this year as one of the  Economic Community of West African States (ECOWAS) Delegates accredited by the Korea Trade Promotion Agency (KOTRA) office to attend your  Trade Missions-the Korean International Commercial and Special Vehicle show tagged "CSV SHOW 2006" held there between Wednesday the 29th of March to Saturday Ist April 2006.
- I **got your contact from** a close Associate of mine who works with the Nigerian Chamber of Commerce and Industry  who visited your country for an International Trade Fair upon my quest for a trusted and reliable foreign businessman or company. 
- I GUESS THIS LETTER MAY COME TO YOU AS SURPRISE SINCE I **HAD NO PREVIOUS CORRESPONDENCE WITH YOU**.
- I know this letter will come to you as surprise. **I managed to get your contact line from** the Cote Divoire Chamber of Commerce, hence I did not waste time to contact you on a business proposal. 


In [10]:
import json
import os

PATH = '../data/separated by email/'

for email in os.listdir(PATH):
    j = json.load(open(os.path.join(PATH, email)))
    sampletext = j['X-TIKA:content'].strip()
    break

In [ ]:
from flashtext import KeywordProcessor
keyword_processor = KeywordProcessor()
keyword_processor.add_keyword('Big Apple', 'New York')

### ix. Attacker estimated age. You will use the USC Data Science AgePredictor, here: https://github.com/USCDataScience/AgePredictor